## Get drug-gene interactions from DGIdb for Floroquinolones

In [1]:
# imports
import sys
import argparse
import json
import requests
import pandas as pd

**Define API, loosely based on official DGIdb-api**

In [28]:
class DGIAPI:
    'API Example class for DGI API.'
    domain = 'http://dgidb.org/'
    api_path = '/api/v2/interactions.json'
    def __init__(self, drugs, raw = False):
        self.drugs = drugs
        self.raw = raw
        
    def run_workflow(self):
        self.create_request()
        self.post_request()
        if(self.raw): # print raw response form server.
            self.raw_response()
            
        self.print_response() # processes the raw json from server
        return self.df

    def create_request(self):
        self.request = "http://dgidb.org/api/v1/interactions.json?genes=FLT1&drug_types=antineoplastic&interaction_sources=TALC"
        self.payload = {}
        if(self.drugs):
            self.payload['drugs'] = self.drugs

    def post_request(self):
        self.request = DGIAPI.domain + DGIAPI.api_path
        self.response = requests.post(self.request, data = self.payload)
        
    def raw_response(self):
        print(json.loads(self.response.content))
        print('\n')
        
    def print_response(self):
        response = json.loads(self.response.content)
        matches = response['matchedTerms']
        
        cols = ['match',
                'searchTerm',
                'drugName',
                'geneID',
                'geneName',
                'longGeneName',
                'interaction']
        lst = []
        if(matches):
            #print("MATCH FOUND") #\t interaction_type \t source")
            for match in matches:  
                drug = match['searchTerm']
                for interaction in match['interactions']:
                    lst.append(['TRUE',
                                match['searchTerm'],
                                match['drugName'],
                                interaction['geneEntrezId'],
                                interaction['geneName'],
                                interaction['geneLongName'],
                                interaction['interactionTypes']]
                                )
                    
                    #print(df_d)
                    #print('\n')
            self.df = pd.DataFrame(lst, columns = cols)
            
        unmatches = response['unmatchedTerms']
        if(unmatches):
            for unmatched in response['unmatchedTerms']:
                lst.append(['FALSE', unmatched, 0, 0, 0, 0, 0])
                self.df = pd.DataFrame(lst, columns = cols)
                
        for ambigious in response['ambiguousTerms']:
            lst.append(['ambigious', ambigious, 0, 0, 0, 0, 0])
            self.df = pd.DataFrame(lst, columns = cols)
        #    print "Unmatched search term: " + unmatched['searchTerm']
            #print "Possible suggestions: " + ",".join(unmatched['suggestions'])



In [10]:
floroq = {
    'DB05488': 'Technetium Tc-99m ciprofloxacin',
    'DB11491': 'Sarafloxacin',
    'DB00365': 'Grepafloxacin',
    'DB00467': 'Enoxacin',
    'DB00487': 'Pefloxacin',
    'DB00685': 'Trovafloxacin',
    'DB00779': 'Nalidixic Acid',
    'DB00817': 'Rosoxacin',
    'DB00827': 'Cinoxacin',
    'DB01044': 'Gaitfloxacin',
    'DB01059': 'Norfloxacin',
    'DB01137': 'Levofloxacin',
    'DB01155': 'Gemifloxacin',
    'DB01405': 'Temafloxacin',
    'DB04576': 'Fleroxacin',
    'DB06160': 'Garenoxacin',
    'DB06600': 'Nemonoxacin',
    'DB06771': 'Besifloxacin',
    'DB08972': 'Flumequine',
    'DB11774': 'Pazufloxacin',
    'DB11892': 'Prulifloxacin',
    'DB13271': 'Sitafloxacin',
    'DB13627': 'Oxolinic acid',
    'DB13772': 'Rufloxacin',
    'DB00218': 'Moxifloxacin',
    'DB00537': 'Ciprofloxacin',
    'DB00978': 'Lomefloxacin',
    'DB01165': 'Ofloxacin',
    'DB01208': 'Sparfloxacin',
    'DB11943': 'Delafloxacin'
    
}

In [11]:
    da = DGIAPI(drugs = 'DB00537', raw = True)
    d = da.run_workflow()

{u'unmatchedTerms': [], u'matchedTerms': [{u'searchTerm': u'DB00537', u'chemblId': u'CHEMBL8', u'interactions': [{u'geneName': u'BAX', u'score': 3, u'pmids': [10741713, 12632069], u'geneEntrezId': 581, u'geneLongName': u'BCL2 ASSOCIATED X, APOPTOSIS REGULATOR', u'sources': [u'NCI'], u'interactionTypes': [], u'interactionId': u'f1bfc557-66c4-47b5-83f5-070723928356'}, {u'geneName': u'TOP2B', u'score': 2, u'pmids': [10592235], u'geneEntrezId': 7155, u'geneLongName': u'DNA TOPOISOMERASE II BETA', u'sources': [u'DrugBank'], u'interactionTypes': [], u'interactionId': u'384abb96-27ce-43ed-82ad-9feae4be9c4b'}, {u'geneName': u'CYP2E1', u'score': 2, u'pmids': [15194011], u'geneEntrezId': 1571, u'geneLongName': u'CYTOCHROME P450 FAMILY 2 SUBFAMILY E MEMBER 1', u'sources': [u'NCI'], u'interactionTypes': [], u'interactionId': u'c4a169b1-4df5-4c6a-a378-11ce0935d260'}], u'drugName': u'CIPROFLOXACIN'}], u'ambiguousTerms': []}




In [12]:
df = pd.DataFrame() 
for drug in floroq:
    da = DGIAPI(drugs = drug)
    df1 = da.run_workflow()
    df = df.append(df1)

In [13]:
df

,match,searchTerm,drugName,geneID,geneName,longGeneName,interaction
0,FALSE,DB06160,0,0,0,0,0
0,TRUE,DB04576,FLEROXACIN,7153,TOP2A,DNA TOPOISOMERASE II ALPHA,[inhibitor]
1,TRUE,DB04576,FLEROXACIN,7155,TOP2B,DNA TOPOISOMERASE II BETA,[inhibitor]
0,TRUE,DB01165,OFLOXACIN,7153,TOP2A,DNA TOPOISOMERASE II ALPHA,[inhibitor]
1,TRUE,DB01165,OFLOXACIN,6539,SLC6A12,SOLUTE CARRIER FAMILY 6 MEMBER 12,[inhibitor]
2,TRUE,DB01165,OFLOXACIN,2936,GSR,GLUTATHIONE-DISULFIDE REDUCTASE,[]
3,TRUE,DB01165,OFLOXACIN,4363,ABCC1,ATP BINDING CASSETTE SUBFAMILY C MEMBER 1,[]
4,TRUE,DB01165,OFLOXACIN,3558,IL2,INTERLEUKIN 2,[]
5,TRUE,DB01165,OFLOXACIN,3553,IL1B,INTERLEUKIN 1 BETA,[]
6,TRUE,DB01165,OFLOXACIN,6529,SLC6A1,SOLUTE CARRIER FAMILY 6 MEMBER 1,[inhibitor]


In [12]:
df.to_csv('drug_interactions.csv')

## BETA LACTAMASES

In [19]:
beta_lactams = pd.read_csv('../data/betalactams.csv', sep = ';')

In [20]:
beta_lactams

,drugID,drugName
0,DB14107,7-Aminodesacetoxycephalosporanic acid
1,DB01163,Amdinocillin
2,DB01060,Amoxicillin
3,DB00415,Ampicillin
4,DB08795,Azidocillin
5,DB01061,Azlocillin
6,DB00355,Aztreonam
7,DB01602,Bacampicillin
8,DB01053,Benzylpenicillin
9,DB00895,Benzylpenicilloyl Polylysine


In [29]:
df = pd.DataFrame() 
for drug in beta_lactams['drugID']:
    da = DGIAPI(drugs = drug)
    df1 = da.run_workflow()
    df = df.append(df1)

In [30]:


da = DGIAPI(drugs = 'DB00417', raw = "TRUE")
da.run_workflow()

{u'unmatchedTerms': [], u'matchedTerms': [], u'ambiguousTerms': [{u'searchTerm': u'DB00417', u'chemblId': u'CHEMBL615', u'interactions': [], u'drugName': u'PENICILLIN V'}, {u'searchTerm': u'DB00417', u'chemblId': u'CHEMBL1200852', u'interactions': [{u'geneName': u'NR1I2', u'score': 1, u'pmids': [], u'geneEntrezId': 8856, u'geneLongName': u'NUCLEAR RECEPTOR SUBFAMILY 1 GROUP I MEMBER 2', u'sources': [u'PharmGKB'], u'interactionTypes': [], u'interactionId': u'7985df88-950d-4900-b334-b13da41c8ad7'}], u'drugName': u'PENICILLIN V POTASSIUM'}]}




,match,searchTerm,drugName,geneID,geneName,longGeneName,interaction
0,ambigious,"{u'searchTerm': u'DB00417', u'chemblId': u'CHE...",0,0,0,0,0
1,ambigious,"{u'searchTerm': u'DB00417', u'chemblId': u'CHE...",0,0,0,0,0


In [32]:
df.to_csv('../data/beta.csv')